In [5]:
import pandas as pd
import numpy as np

In [6]:
import os
os.chdir('./')

In [7]:
sabdab = pd.read_csv('./sabdab_CH_filtered_v1_24-03-04.csv', index_col=0)

In [8]:
sabdab['pdb_id'] = sabdab['PDB-ID_HC_LC_AG.1'].apply(lambda x: x.split('_')[0].lower())

In [9]:
print(len(sabdab))

# There were not many of these, but these were complexes with a lot of unknowns
sabdab= sabdab[sabdab['AG_AA'].apply(lambda x: x.count('X') < 2)]
print(len(sabdab))

2533
2519


In [10]:
# Dropping complex antigens - currently not sure how to handle these
print(len(sabdab))
sabdab = sabdab[~sabdab['AG_AA'].apply(lambda x: '|' in x)]
print(len(sabdab))

2519
2258


In [11]:
X_ant = sabdab['antigen_noSignalP']

In [12]:
his_tag = 'GSHHHHHHGS'
his_tag6 ='HHHHHHH'
strep_tag = 'WSHQPFEK'
strep_tagII = 'WSHPQFEK'
strep_tagIII = 'SAWSHPQFEK'
flexible_connector = 'GGGSGGGGSGGS'
twin_strep = 'SAWSHPQFEKGGGSGGGGSGGSAWSHPQFEK'
thrombin = 'LVPRGS'
full_thrombin = 'MGSSHHHHHHSSGLVPRGSHM'

spytag001 = 'AHIVMVDAYKPTK'
spytag003 = 'RGVPHIVMVDAYKRYK'

myc_tag = 'EQKLISEEDL'
flagtag = 'DYKDDDDK'
HAtag = 'YPYDVPDYA'

# https://www.uniprot.org/uniprotkb/O55244/entry#sequences
cloning_site = 'MGSSHHHHHHSSGLVPRGSHMISNSSLVPLALVSSPVSSGRGSHHHHHHHHHH'


# mbp = 'MKIKTGARILALSALTTMMFS
# ASALAKIEEGKLVIWINGDKGYNGLAEVGKKFEKDTGIKVTVEHPDKLEEKFPQVAATGDGPDIIFWAHDRFGGYAQSGLLAEITPDKAFQDKLYPFTWDAVRYNGKLIAYPIAVEALSLIYNKDLLPNPPKTWEEIPALDKELKAKGKSALMFNLQEPYFTWPLIAADGGYAFKYENGKYDIKDVGVDNAGAKAGLTFLVDLIKNKHMNADTDYSIAEAAFNKGETAMTINGPWAWSNIDTSKVNYGVTVLPTFKGQPSKPFVGVLSAGINAASPNKELAKEFLENYLLTDEGLEAVNKDKPLGAVALKSYEEELAKDPRIAATMENAQKGEIMPNIPQMSAFWYAVRTAVINAASGRQTVDEALKDAQTRITK'

trimerization_domain = 'YIPEAPRDGQAYVRKDGEWVLLSTFLG'


# avi_tag = 'GSGYIPEAPRDGQAYVRKDGEWVLLSTFLGLEVLFQGPGGLNDIFEAQKIEWHE'
# idk = 'MKAKLLVLLCTFTATYA' # maybe foldon?
hrv3c = 'LEVLFQGP'
avi_tag = 'GLNDIFEAQKIEWHE'

tags = [his_tag, his_tag6, strep_tag, strep_tagII, strep_tagIII, flexible_connector, avi_tag, hrv3c, twin_strep, thrombin, 
        spytag001, spytag003, myc_tag, flagtag, HAtag, trimerization_domain, full_thrombin]

In [13]:
tagged_dfs = {}
for tag in tags:
    df = X_ant[X_ant.apply(lambda x: tag in x)]
    if len(df) > 0:

        tagged_dfs[tag] = df

In [14]:
tagged_dfs.keys()

dict_keys(['GSHHHHHHGS', 'HHHHHHH', 'WSHPQFEK', 'SAWSHPQFEK', 'GGGSGGGGSGGS', 'GLNDIFEAQKIEWHE', 'LEVLFQGP', 'SAWSHPQFEKGGGSGGGGSGGSAWSHPQFEK', 'LVPRGS', 'EQKLISEEDL', 'DYKDDDDK', 'YPYDVPDYA', 'YIPEAPRDGQAYVRKDGEWVLLSTFLG', 'MGSSHHHHHHSSGLVPRGSHM'])

In [15]:
# This works
X_ant.loc[tagged_dfs[full_thrombin].index] = tagged_dfs[full_thrombin].apply(lambda x: x.replace(full_thrombin, ''))

In [16]:
flag1 = 'MKHHHHHHHHHHSSDYKDDDDK'
flag2 = 'MHHHHHHHHDYKDDDDK'
flag3 = 'MGSSHHHHHHSDYKDDDDK'

In [17]:
tagged_dfs[flagtag] = tagged_dfs[flagtag].apply(lambda x: x.replace(flag1, '') if flag1 in x else x)
tagged_dfs[flagtag]  = tagged_dfs[flagtag].apply(lambda x: x.replace(flag2, '') if flag2 in x else x)
tagged_dfs[flagtag]  = tagged_dfs[flagtag].apply(lambda x: x.replace(flag3, '') if flag3 in x else x)
tagged_dfs[flagtag]  = tagged_dfs[flagtag].apply(lambda x: x.replace(flagtag, '') if flagtag in x else x)

In [18]:
X_ant.loc[tagged_dfs[flagtag].index] = tagged_dfs[flagtag]

C:\Users\perry\AppData\Local\Temp\ipykernel_15412\996960093.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_ant.loc[tagged_dfs[flagtag].index] = tagged_dfs[flagtag]


In [19]:
def find_and_replace_tag(seq, tag):
    seq_len = len(seq)
    split_seq = seq.split(tag)

    if len(split_seq) == 2:

        if len(split_seq[0]) < 30:
            print(split_seq[0])
            return split_seq[1]
        
        if len(split_seq[1]) < 30:
            print(split_seq[1])
            return split_seq[0]
        
        else:
            print('long attachment:')
            print(seq)
            print('---')
            
            return seq
        
    if len(split_seq) >2:
        print('>2 splits')
        return True
    
    if len(split_seq) == 1:
        return seq
    

In [20]:
# Removing his tags from beginning of these
# X_ant.loc[tagged_dfs[his_tag].index] = tagged_dfs[his_tag].apply(lambda x: x.split(his_tag)[1])

# X_ant.loc[tagged_dfs[his_tag6].index] = tagged_dfs[his_tag6].apply(lambda x: x.split(his_tag6)[1])

# X_ant = X_ant.apply(lambda x: fine_and_replace_tag(x, his_tag))
test = X_ant.apply(lambda x: find_and_replace_tag(x, 'HHHHHHH'))
X_ant = test[test != True]

H
H
H
HHH
YKDDDDK
H
YKDDDDK
H
H
H

H
H
H
H

H
HSAWSHPQFEK
HSAWSHPQFEK
HSAWSHPQFEK
HSAWSHPQFEK
HSAWSHPQFEK
HSAWSHPQFEK
HSAWSHPQFEK
HSAWSHPQFEK
HHH
APMAEGGGQN
APMAEGGGQN
APMAEGGGQN
APMAEGGGQN
HHH
H
MA
HHH
H
H
H
H
HHHWSHPQFEK
HSAWSHPQFEK
HSAWSHPQFEK
HSAWSHPQFEK
H
HSAWSHPQFEK
H
long attachment:
DLDRCTTFDDVQAPNYTQHTSSMRGVYYPDEIFRSDTLYLTQDLFLPFYSNVTGFHTINHTFGNPVIPFKDGIYFAATEKSNVVRGWVFGSTMNNKSQSVIIINNSTNVVIRACNFELCDNPFFAVSKPMGTQTHTMIFDNAFNCTFEYISDAFSLDVSEKSGNFKHLREFVFKNKDGFLYVYKGYQPIDVVRDLPSGFNTLKPIFKLPLGINITNFRAILTAFSPAQDIWGTSAAAYFVGYLKPTTFMLKYDENGTITDAVDCSQNPLAELKCSVKSFEIDKGIYQTSNFRVVPSGDVVRFPNITNLCPFGEVFNATKFPSVYAWERKKISNCVADYSVLYNSTFFSTFKCYGVSATKLNDLCFSNVYADSFVVKGDDVRQIAPGQTGVIADYNYKLPDDFMGCVLAWNTRNIDATSTGNYNYKYRYLRHGKLRPFERDISNVPFSPDGKPCTPPALNCYWPLNDYGFYTTTGIGYQPYRVVVLSFELLNAPATVCGPKLSTDLIKNQCVNFNFNGLTGTGVLTPSSKRFQPFQQFGRDVSDFTDSVRDPKTSEILDISPCSFGGVSVITPGTNASSEVAVLYQDVNCTDVSTAIHADQLTPAWRIYSTGNNVFQTQAGCLIGAEHVDTSYECDIPIGAGICASYHTVSLLRSTSQKSIVAYTMSLGADSSIAYSNNTIAIPTNFSISITTEVMPVSMAKTSVDCNM

A lot of these have long tags. Will try removing histines again at the end.

In [21]:
tagged_dfs = {}
for tag in tags:
    df = X_ant[X_ant.apply(lambda x: tag in x)]
    if len(df) > 0:

        tagged_dfs[tag] = df

In [22]:
tagged_dfs.keys()

dict_keys(['GSHHHHHHGS', 'HHHHHHH', 'WSHPQFEK', 'SAWSHPQFEK', 'GGGSGGGGSGGS', 'GLNDIFEAQKIEWHE', 'LEVLFQGP', 'SAWSHPQFEKGGGSGGGGSGGSAWSHPQFEK', 'LVPRGS', 'EQKLISEEDL', 'YPYDVPDYA', 'YIPEAPRDGQAYVRKDGEWVLLSTFLG'])

In [23]:
test = X_ant.apply(lambda x: find_and_replace_tag(x, 'GSHHHHHHGS'))
X_ant = test[test != True]

MR


In [24]:
test = X_ant.apply(lambda x: find_and_replace_tag(x, 'GGGSGGGGSGGS'))
X_ant = test[test != True]

AWSHPQFEK
AWSHPQFEK
AWSHPQFEK
AWSHPQFEK
AWSHPQFEK
AWSHPQFEK
AWSHPQFEK
AWSHPQFEK
AWSHPQFEK
AWSHPQFEK
AWSHPQFEKGS
AWSHPQFEKGS
AWSHPQFEK
AWSHPQFEK
AWSHPQFEK
AWSHPQFEK
AWSHPQFEK
AWSHPQFEK
AWSHPQFEK
AWSHPQFEK
AWSHPQFEK
AWSHPQFEK
AWSHPQFEK
AWSHPQFEK
AWSHPQFEK
AWSHPQFEK
AWSHPQFEK
AWSHPQFEK
AWSHPQFEK
AWSHPQFEK
AWSHPQFEK
AWSHPQFEK
AWSHPQFEK
AWSHPQFEK
AWSHPQFEK
AWSHPQFEK
AWSHPQFEK
AWSHPQFEK
AWSHPQFEK
AWSHPQFEK
AWSHPQFEK
AWSHPQFEK
AWSHPQFEK
AWSHPQFEK
AWSHPQFEK
AWSHPQFEK
AWSHPQFEK
AWSHPQFEK
AWSHPQFEK
AWSHPQFEK
AWSHPQFEK
AWSHPQFEK
AWSHPQFEK
AWSHPQFEK
AWSHPQFEK
AWSHPQFEK
AWSHPQFEK
AWSHPQFEKGS
AWSHPQFEKGS
AWSHPQFEKGS
AWSHPQFEKGS
AWSHPQFEK
AWSHPQFEK
AWSHPQFEK
AWSHPQFEK
AWSHPQFEK
AWSHPQFEK
AWSHPQFEK
AWSHPQFEK
AWSHPQFEK
AWSHPQFEK
AWSHPQFEK
AWSHPQFEK
AWSHPQFEK
AWSHPQFEK
AWSHPQFEK
AWSHPQFEK
AWSHPQFEK
AWSHPQFEK
AWSHPQFEKGS
AWSHPQFEK
AWSHPQFEK
AWSHPQFEK
AWSHPQFEK


In [25]:
def find_and_replace_tag(seq, tag):
    seq_len = len(seq)
    split_seq = seq.split(tag)

    if len(split_seq) == 2:

        if len(split_seq[0]) < 30:
            print('Removed: ' + split_seq[0])
            return split_seq[1]
        
        if len(split_seq[1]) < 30:
            print('Removed: ' + split_seq[1])
            return split_seq[0]
        
        else:
            print('long attachment for: ' + seq)
            return seq
        
    if len(split_seq) >2:
        print('>2 splits')
        return seq
    
    if len(split_seq) == 1:
        return seq
    

In [26]:
# These are on end, so entire thing is removed
X_ant = X_ant.apply(lambda x: find_and_replace_tag(x, 'WSHPQFEKGGGSGGGSGGGSWSHPQFEK'))

Removed: 
Removed: 
Removed: 
Removed: GA
Removed: MGS
Removed: 
Removed: 
Removed: 
Removed: 
Removed: 


In [27]:
tagged_dfs = {}
for tag in tags:
    df = X_ant[X_ant.apply(lambda x: tag in x)]
    if len(df) > 0:

        tagged_dfs[tag] = df

In [28]:
tagged_dfs.keys()

dict_keys(['HHHHHHH', 'WSHPQFEK', 'SAWSHPQFEK', 'GLNDIFEAQKIEWHE', 'LEVLFQGP', 'LVPRGS', 'EQKLISEEDL', 'YPYDVPDYA', 'YIPEAPRDGQAYVRKDGEWVLLSTFLG'])

In [29]:
X_ant = X_ant.apply(lambda x: find_and_replace_tag(x, 'SAWSHPQFEKGGSGGGSGGSAWSHPQFEK'))

Removed: 
Removed: 
Removed: 
Removed: 


In [30]:
test = X_ant.apply(lambda x: find_and_replace_tag(x, 'SAWSHPQFEK'))

>2 splits
Removed: 
Removed: 
Removed: 
Removed: 
Removed: 
Removed: 
Removed: 
Removed: 
Removed: 
>2 splits
Removed: 
Removed: 
Removed: 
Removed: 
Removed: 
Removed: 
Removed: 
Removed: 
Removed: GGS
Removed: GGS
Removed: GGS
Removed: GGS
Removed: GGS
Removed: GGS
Removed: GGS
Removed: GGS
Removed: GGS
Removed: GGS
Removed: GGS
Removed: GGS
Removed: GGS
Removed: 
Removed: GGS
Removed: GGS
Removed: GGS
Removed: GGS
Removed: GGS
Removed: GGS
Removed: GGS
>2 splits
>2 splits
>2 splits
>2 splits
>2 splits
>2 splits
>2 splits
>2 splits
Removed: 
Removed: 
Removed: 
Removed: 
Removed: 
Removed: 
Removed: 
Removed: 
Removed: 
Removed: 
Removed: 
Removed: 
Removed: 
Removed: 
Removed: 
Removed: 
Removed: 
Removed: 
Removed: 
Removed: 
Removed: 
Removed: 
Removed: 
Removed: 
Removed: 
Removed: 
Removed: 
Removed: 
Removed: 
Removed: 
Removed: 
Removed: 
Removed: 
Removed: 
Removed: 
Removed: 
Removed: 
Removed: 
Removed: 
Removed: 
Removed: 
Removed: 
Removed: 
Removed: 
Removed: 
Removed: 


In [31]:
X_ant = X_ant.apply(lambda x: find_and_replace_tag(x, 'SAWSHPQFEKGGGSGGGSGGGSAWSHPQFEK'))

Removed: 
Removed: 


In [32]:
X_ant = X_ant.apply(lambda x: find_and_replace_tag(x, 'SAWSHPQFEKGGGSGGGSGGSAWSHPQFEK'))

Removed: 
Removed: 
Removed: 
Removed: 
Removed: 
Removed: 
Removed: 
Removed: 
Removed: 
Removed: 
Removed: 
Removed: 
Removed: 
Removed: 
Removed: 
Removed: 
Removed: 


In [33]:
tagged_dfs = {}
for tag in tags:
    df = X_ant[X_ant.apply(lambda x: tag in x)]
    if len(df) > 0:

        tagged_dfs[tag] = df

In [34]:
tagged_dfs.keys()

dict_keys(['HHHHHHH', 'WSHPQFEK', 'SAWSHPQFEK', 'GLNDIFEAQKIEWHE', 'LEVLFQGP', 'LVPRGS', 'EQKLISEEDL', 'YPYDVPDYA', 'YIPEAPRDGQAYVRKDGEWVLLSTFLG'])

In [35]:
X_ant =  X_ant.apply(lambda x: find_and_replace_tag(x, 'EQKLISEEDLGGEQKLISEEDLHHHHH'))

Removed: H
Removed: H
Removed: H
Removed: H
Removed: H


In [36]:
X_ant =  X_ant.apply(lambda x: find_and_replace_tag(x, 'EQKLISEEDLGGEQKLISEEDLSGHHHHHHSS'))

In [37]:
X_ant =  X_ant.apply(lambda x: find_and_replace_tag(x, 'EQKLISEEDL'))

Removed: NLDLVPRGSSSHHHHHHSSG
Removed: NGAPHHHHHH
Removed: NSAVDHHHHHH
Removed: NSAVDHHHHHH
Removed: NSAVDHHHHHH
Removed: NSAVDHHHHHH
Removed: NSAVDHHHHHH
Removed: NSAVDGSGLNDIFEAQKIEWHENLYFQG
Removed: NSAVDGSGLNDIFEAQKIEWHENLYFQG
Removed: NMHTGHHHHHH
Removed: NMHTGHHHHHH
Removed: NMHTGHHHHHH
Removed: NMHTGHHHHHH


In [38]:
tagged_dfs = {}
for tag in tags:
    df = X_ant[X_ant.apply(lambda x: tag in x)]
    if len(df) > 0:

        tagged_dfs[tag] = df

In [39]:
tagged_dfs.keys()

dict_keys(['HHHHHHH', 'WSHPQFEK', 'SAWSHPQFEK', 'GLNDIFEAQKIEWHE', 'LEVLFQGP', 'LVPRGS', 'YPYDVPDYA', 'YIPEAPRDGQAYVRKDGEWVLLSTFLG'])

Note: trimerization domains are primarily in our LIBRA-seq data. not sure what this means

In [40]:
X_ant = X_ant.apply(lambda x: find_and_replace_tag(x, 'DYKDDDDKAADYKDDDDK'))

In [41]:
X_ant = X_ant.apply(lambda x: find_and_replace_tag(x, 'DYKDDDDK'))

In [42]:
tagged_dfs = {}
for tag in tags:
    df = X_ant[X_ant.apply(lambda x: tag in x)]
    if len(df) > 0:

        tagged_dfs[tag] = df

In [43]:
X_ant = X_ant.apply(lambda x: find_and_replace_tag(x, 'LVPRGS'))

Removed: ASWSHPQFEKSGGGGG
Removed: 
Removed: 
Removed: 
Removed: 
Removed: MGSSHHHHHHSSG
Removed: GHHHHHH
long attachment for: SDKICIGYHANNSTDTVDTVLEKNVTVTHSVNLLENKHNGKLCKLRGVAPLHLGKCNIAGWLLGNPECESLATASSWSYIVETSSSNNGTCYPGDFINYEELREQLSSVSSFEKFEIFPKTSSWPNHETNKGVTAACPHAGTNSFYKNLIWLVKKENSYPKINISYTNNRGKEVLVLWAIHHPPTSTDQQSLYQNANSYVFVGSSRYSRKFEPEIATRPKVRGQAGRMNYYWTLVEPGDKITFEATGNLVVPRYAFALKRNSGSGIIISETPVHDCDTTCQTPNGAINTSLPFQNIHPVTIGECPKYVKSTKLRMATGLRNIPSIQSRGLFGAIAGFIEGGWTGMIDGWYGYHHQNEQGSGYAADLKSTQNAIDGITNKVNSVIEKMNTQFTAVGKEFSHLERRIENLNKKVDDGFIDIWTYNAELLVLLENERTLDYHDSNVKTLYEKVRSQLKNNAKEIGNGCFEFYHKCDDTCMESVKNGTYDYPKYSEEFLVPRGSPGSGYIPEAPRDGQAYVRKDGEWVLLSTFLGHHHHHH
long attachment for: SDTICIGYHANNSTDTVDTVLEKNVTVTHSVNLLEDSHNGKLCRLKGTAPLQLGNCSVAGWILGNPECESLFSKESWSYIAETPKPENGTCYPGYFADYEELREQLSSVSSFERFEIFPKESSWPNHTVTKGVTAACSHNGKSSFYKNLLWLTEKNGLYPNLSKSYVNNKEKEVLVLWGVHHPSNIGDQRAIYHTENAYVSVVSSHYSRRFTPEIAKRPKVRGQEGRINYYWTLLEPGDTIIFEANGNLIAPWYAFALSRGFGSGIITSNASMGECDAKCQTPQGAINSSLPFQNVHPVTIGECPKYVRSTKLR

In [44]:
tagged_dfs.keys()

dict_keys(['HHHHHHH', 'WSHPQFEK', 'SAWSHPQFEK', 'GLNDIFEAQKIEWHE', 'LEVLFQGP', 'LVPRGS', 'YPYDVPDYA', 'YIPEAPRDGQAYVRKDGEWVLLSTFLG'])

In [45]:
# Removing 3 from above 
X_ant = X_ant[~X_ant.apply(lambda x: type(x) == int)]

In [46]:
# GSGGLNDIFEAQKIEWHE is avitag (see methods here: https://assets.researchsquare.com/files/rs-3045457/v1_covered_729e744e-72c6-45d9-9170-a969cd8b1953.pdf?c=1688438804)
X_ant = X_ant.apply(lambda x: find_and_replace_tag(x, 'LEVLFQGP'))

Removed: GAAEDQVDPRLIDGK
Removed: 
Removed: 
Removed: GHHHHHH
long attachment for: QVCTGTDMKLRLPASPETHLDMLRHLYQGCQVVQGNLELTYLPTNASLSFLQDIQEVQGYVLIAHNQVRQVPLQRLRIVRGTQLFEDNYALAVLDNGDPLNNTTPVTGASPGGLRELQLRSLTEILKGGVLIQRNPQLCYQDTILWKDIFHKNNQLALTLIDTNRSRACHPCSPMCKGSRCWGESSEDCQSLTRTVCAGGCARCKGPLPTDCCHEQCAAGCTGPKHSDCLACLHFNHSGICELHCPALVTYNTDTFESMPNPEGRYTFGASCVTACPYNYLSTDVGFCTLVCPLHNQEVTAEDGTQRCEKCSKPCARVCYGLGMEHLREVRAVTSANIQEFAGCKKIFGSLAFLPESFDGDPASNTAPLQPEQLQVFETLEEITGYLYISAWPDSLPDLSVFQNLQVIRGRILHNGAYSLTLQGLGISWLGLRSLRELGSGLALIHHNTHLCFVHTVPWDQLFRNPHQALLHTANRPEDECVGEGLACHQLCARGHCWGPGPTQCVNCSQFLRGQECVEECRVLQGLPREYVNARHCLPCHPECQPQNGSVTCFGPEADQCVACAHYKDPPFCVARCPSGVKPDLSYMPIWKFPDEEGACQPCPINCTHSCVDLDDKGCPAEQRASPLTSIISAVVGILLVVVLGVVFGILIKRRQQKIRKYTMRRLLQETELVEPLTPSGAMPNQAQMRILKETELRKVKVLGSGAFGTVYKGIWIPDGENVKIPVAIKVLRENTSPKANKEILDEAYVMAGVDSPYVSRLLGICLTSTVQLVTQLMPYGCLLDHVRENRGRLGSQDLLNWCMQIAKGMSYLEDVRLVHRDLAARNVLVKSPNHVKITDFGLARLLDIDETEYHADGGKVPIKWMALESILRRRFTHQSDVWSYGVTVWELMTFGAKPYDGIPAREIPDLLEKGER

In [47]:
tagged_dfs = {}
for tag in tags:
    df = X_ant[X_ant.apply(lambda x: tag in x)]
    if len(df) > 0:

        tagged_dfs[tag] = df

tagged_dfs.keys()

dict_keys(['WSHPQFEK', 'SAWSHPQFEK', 'GLNDIFEAQKIEWHE', 'LEVLFQGP', 'LVPRGS', 'YPYDVPDYA', 'YIPEAPRDGQAYVRKDGEWVLLSTFLG'])

In [48]:
X_ant = X_ant.apply(lambda x: find_and_replace_tag(x, 'WSHPQFEK'))
X_ant = X_ant.apply(lambda x: find_and_replace_tag(x, 'SAWSHPQFEK'))

>2 splits
Removed: 
Removed: 
Removed: 
Removed: 
Removed: 
Removed: 
Removed: 
Removed: 
>2 splits
>2 splits
>2 splits
>2 splits
>2 splits
>2 splits
>2 splits
>2 splits
>2 splits
>2 splits
>2 splits
>2 splits
>2 splits
Removed: 
>2 splits
>2 splits
>2 splits
>2 splits
>2 splits
>2 splits
>2 splits
>2 splits
>2 splits
>2 splits
>2 splits
>2 splits
Removed: GGS
Removed: GGS
Removed: GGS
Removed: GGS
Removed: GGS
Removed: GGS
Removed: GGS
Removed: GGS
Removed: GGS
Removed: GGS
Removed: GGS
Removed: GGS
Removed: GGS
Removed: GGS
Removed: GGS
Removed: GGS
Removed: GGS
Removed: GGS
Removed: GGS
Removed: GGS
Removed: 
Removed: 
Removed: 
Removed: 


In [49]:
tagged_dfs = {}
for tag in tags:
    df = X_ant[X_ant.apply(lambda x: tag in x)]
    if len(df) > 0:

        tagged_dfs[tag] = df

tagged_dfs.keys()

dict_keys(['WSHPQFEK', 'SAWSHPQFEK', 'GLNDIFEAQKIEWHE', 'LEVLFQGP', 'LVPRGS', 'YPYDVPDYA', 'YIPEAPRDGQAYVRKDGEWVLLSTFLG'])

In [50]:
X_ant = X_ant.apply(lambda x: x.replace('SAWSHPQFEKGGGSGGGSGGSSAWSHPQFEK', ''))

In [51]:
test = X_ant.apply(lambda x: find_and_replace_tag(x, 'HHHHH'))
X_ant = test[test != True]

Removed: H
Removed: H
Removed: H
Removed: H
Removed: H
Removed: H
Removed: H
Removed: M
Removed: H
Removed: H
Removed: 
Removed: H
Removed: H
Removed: H
Removed: MA
Removed: 
Removed: HALE
Removed: MRGS
Removed: MRGS
Removed: H
Removed: H
Removed: H
Removed: H
Removed: H
Removed: MGSS
Removed: 
Removed: H
Removed: H
Removed: H
Removed: H
Removed: H
Removed: H
Removed: 
Removed: 
Removed: 
Removed: H
Removed: H
Removed: H
Removed: H
Removed: H
Removed: H
Removed: H
Removed: HGLNDIFEAQKIEWHE
Removed: H
Removed: H
Removed: H
Removed: H
Removed: H
Removed: H
Removed: 
Removed: APE
Removed: H
Removed: H
Removed: H
Removed: H
Removed: MGSS
Removed: H
Removed: HLNDIFEAQKIEWHE
Removed: H
Removed: H
Removed: H
Removed: H
Removed: H
Removed: H
Removed: H
Removed: H
Removed: H
Removed: H
Removed: H
Removed: H
Removed: H
Removed: H
Removed: H
Removed: H
Removed: ADP
Removed: ADP
Removed: 
Removed: 
Removed: 
Removed: 
Removed: 
Removed: 
Removed: H
Removed: H
Removed: 
Removed: ADP
Removed: ADP
Re

Looking pretty clean now. A few tags may have escaped, but there are only 1 example of each. Will not allow model to target these when trying to generalize.

In [52]:
X_ant

PDB-ID_HC_LC_AG
8AON_B_C_A    GSHMSALYSPSDPLTLLQADTVRGAVLGSRSAWAVEFFASWCGHCI...
4XWO_C_D_A    MDLTVEPNLHSLITSTTHKWIFVGGKGGVGKTTSSCSIAIQMALSQ...
6VGR_C_D_A    TTPGAPRALSTLGSPSLFTTPGVPSALTTPGLTTPGTPKTLDLRGR...
6RCU_B_C_A    ENAIKKTKNQENQLTLLPIKSTEEEKDDIKNGKDIKKEIDNDKENI...
6RCU_E_D_A    ENAIKKTKNQENQLTLLPIKSTEEEKDDIKNGKDIKKEIDNDKENI...
                                    ...                        
6MVL_H_L_A    AFKVATPYSLYVCPEGQNVTLTCRLLGPVDKGHDVTFYKTWYRSSR...
3MXW_H_L_A    GFGKRRHPKKLTPLAYKQFIPNVAEKTLGASGRYEGKISRNSERFK...
6IAP_E_D_A    TLPKPFIWAEPHFMVPKEKQVTICCQGNYGAVEYQLHFEGSLFAVD...
6IAP_H_L_A    TLPKPFIWAEPHFMVPKEKQVTICCQGNYGAVEYQLHFEGSLFAVD...
8PE9_H_L_A    RDGLLSYTAPVGQTMYLSEAVYLNDSTYDGHTVGGLQYGGLGQLAD...
Name: antigen_noSignalP, Length: 2258, dtype: object

In [53]:
sabdab['detagged_ant_seq'] = X_ant

In [66]:
sabdab.to_csv('sabdab_detagged_24-03-12.csv')